# Homework: week 1



# Q1. Running Elastic Search

In [1]:
%%bash

curl localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


{
  "name" : "f8dc1bfb1b07",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "A0X8sWgLRnSCq5lyDpdEJw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


100   539  100   539    0     0  36804      0 --:--:-- --:--:-- --:--:-- 41461


The version.build_hash value is: 42f05b9372a9a4a470db3b52817899b99a76ee73

### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [4]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'f8dc1bfb1b07', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'A0X8sWgLRnSCq5lyDpdEJw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # Allowsto filter
        }
    }
}

index_name = "course-questions-index"
response = es_client.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-index/EFvTHOUpTjGZEaCajVCVlQ] already exists')

In [6]:
# We bring the documents to elastic search using tqdm progress bar
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

c:\Users\User\Documents\llm-zoomcamp-dbeta95\llm-zoomcamp-3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:47<00:00, 19.80it/s]


As we can see above, we used the method "index" from the ElasticSearch client to add our data.

## Q3. Searching

In [6]:
query = "How do I execute a command in a running docker container?"


search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [7]:
response['hits']

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions-index',
   '_id': 'wn0RWJABJacXeW926svw',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions-index',
   '_id': '0H0RWJABJacXeW92iMmA',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course f

The top ranking result has an score of 84.05

## Q4. Filtering

In [8]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter":{
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [9]:
response['hits']['hits'][2]

{'_index': 'course-questions-index',
 '_id': '4n0RWJABJacXeW928csv',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

As shown above the question is 'question': 'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [10]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for hit in response['hits']['hits']:
    question = hit['_source']['question']
    text = hit['_source']['text']
    context += context_template.format(question=question, text=text).strip() + "\n\n"

In [11]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()

In [13]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [14]:
len(prompt)

1462

## Q6.Tokens



In [15]:
import tiktoken

In [16]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [17]:
encoded_prompt = encoding.encode(prompt)

In [19]:
len(encoded_prompt)

322

## Bonus: generating the answer

The LLM will be replaced with gemini using Vertex, since I don't have a paid suscription to GPT

In [24]:
import os
import vertexai
from vertexai.generative_models import GenerativeModel
from dotenv import load_dotenv

load_dotenv()

PROJECT_ID = os.getenv('PROJECT_ID')
REGION = os.getenv('REGION')

In [29]:
vertexai.init(
    project = PROJECT_ID,
    location = REGION
)
def llm(prompt):
    
    vertex_llm = GenerativeModel("gemini-1.5-pro-001")

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0,
        "top_p": 0.95,
    }

    responses = vertex_llm.generate_content(
        prompt,
        generation_config=generation_config,
        stream=True,
    )

    output = ""
    for response in responses:
        output += response.text + "\n"

    return output

In [30]:
output = llm(prompt)

In [33]:
print(output)

1
. Use `docker ps` command to find the ID of the running container.

2. Run `docker exec -it <container-id> bash`
 to execute bash in the container. Replace `<container-id>` with the actual container ID. 





In [34]:
len(output)

208

In [37]:
encoded_output = encoding.encode(output)

In [38]:
len(encoded_output)

51

## Bonus: calculating the costs (ungraded)

In [35]:
def openai_price(requests, input_tokens, output_tokens):
    inputs_price = requests*input_tokens*(5/1000000)
    outputs_price = requests*output_tokens*(15/1000000)
    return inputs_price+outputs_price

In [39]:
num_requests = 1000
average_input_tokens = 150
average_output_tokens = 250
q6_input_tokens = len(encoded_prompt)
q7_output_tokens = len(encoded_output)

In [40]:
openai_price(num_requests, average_input_tokens, average_output_tokens)

4.5

In [41]:
openai_price(num_requests, q6_input_tokens, q7_output_tokens)

2.375